# VGG16 com Validação Cruzada
Este notebook aplica a rede **VGG16** utilizando os dados carregados pelo `DataLoader`, avaliando o desempenho via **validação cruzada (K-Fold)**.

# Importações Necessárias

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import os
import cv2
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Imports de classes dentro do projeto
from get_data_from_db import DataLoader
from create_model import create_vgg_model


C:\Users\Matheus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Obtendo os dados de treino

In [2]:
# Inicializa o DataLoader
data_loader = DataLoader()

# Carrega os dados de treino com labels
train_data_with_labels = data_loader.get_train_data() #Pega apenas os dados de treino baixados do database
print(f"Total de amostras de treino: {len(train_data_with_labels)}") # Printa o total de amostras de treino apenas

IMG_SIZE = (224, 224)  # padrão para VGG16
X, y = [], [] # Vetores que vão armazenar os dados de treino

# Reconstruindo caminho absoluto corretamente
for filename, label in train_data_with_labels: #Pra cada amostra de treino, adiciona um label para o nome dela, tipo "train_normal_(0).png"
    if label == 0:
        filepath = os.path.join(data_loader.train_normal_path, filename)
    elif label == 1:
        filepath = os.path.join(data_loader.train_ulcerative_colitis_path, filename)
    elif label == 2:
        filepath = os.path.join(data_loader.train_polyps_path, filename)
    elif label == 3:
        filepath = os.path.join(data_loader.train_esophagitis_path, filename)
    else:
        continue

    img = cv2.imread(filepath)
    if img is None:
        print(f"[AVISO] Não foi possível carregar: {filepath}")
        continue

    img = cv2.resize(img, IMG_SIZE)
    X.append(img)
    y.append(label)

X = np.array(X, dtype="float32") / 255.0
y = to_categorical(np.array(y))

print(f"Formato de X: {X.shape}")
print(f"Formato de y: {y.shape}")


Dataset downloaded/cached at: C:\Users\Matheus\.cache\kagglehub\datasets\francismon\curated-colon-dataset-for-deep-learning\versions\1
Total de amostras de treino: 3200
Formato de X: (3200, 224, 224, 3)
Formato de y: (3200, 4)


# Treinamento com Validação Cruzada

In [3]:

#Vai aplicar a validação cruzada K-Fold
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"Treinando fold {fold+1}...")
    
    x_train_cv, x_val_cv = X[train_idx], X[val_idx]
    y_train_cv, y_val_cv = y[train_idx], y[val_idx]
    
    model = create_vgg_model(input_shape=(224, 224, 3), num_classes=y.shape[1])
    model.fit(x_train_cv, y_train_cv, 
              epochs=8, batch_size=32, 
              validation_data=(x_val_cv, y_val_cv), verbose=1)
    
    _, acc = model.evaluate(x_val_cv, y_val_cv, verbose=0)
    accuracies.append(acc)
    
print("Acurácias por fold:", accuracies)
print("Acurácia média:", np.mean(accuracies))


Treinando fold 1...
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/8
 3/80 ━━━━━━━━━━━━━━━━━━━━ 3:17 3s/step - accuracy: 0.2726 - loss: 4.0431

KeyboardInterrupt: 

# Testando

In [ ]:
# Pega todos os arquivos de teste com labels
test_files_with_labels = data_loader.get_test_data()
print(f"Total de amostras de teste: {len(test_files_with_labels)}")

IMG_SIZE = (224, 224)
X_test, y_test = [], []

# Carrega e processa as imagens de teste
for filename, label in test_files_with_labels:
    if label == 0:
        filepath = os.path.join(data_loader.test_normal_path, filename)
    elif label == 1:
        filepath = os.path.join(data_loader.test_ulcerative_colitis_path, filename)
    elif label == 2:
        filepath = os.path.join(data_loader.test_polyps_path, filename)
    elif label == 3:
        filepath = os.path.join(data_loader.test_esophagitis_path, filename)
    else:
        continue

    img = cv2.imread(filepath)
    if img is None:
        print(f"[AVISO] Não foi possível carregar: {filepath}")
        continue

    img = cv2.resize(img, IMG_SIZE)
    X_test.append(img)
    y_test.append(label)

# Normaliza e converte para categorical
X_test = np.array(X_test, dtype="float32") / 255.0
y_test = to_categorical(np.array(y_test))

print(f"Formato de X_test: {X_test.shape}")
print(f"Formato de y_test: {y_test.shape}")

# Avalia o modelo já treinado
loss, acc = model.evaluate(X_test, y_test, verbose=1)
print(f"[TESTE FINAL] Loss: {loss:.4f}, Acurácia: {acc:.4f}")


# Grafico de Matriz de Confusão

In [ ]:
# Converte y_test e predições de volta para labels inteiros
y_true = np.argmax(y_test, axis=1)
y_pred = np.argmax(predicoes, axis=1)

# Cria matriz de confusão
cm = confusion_matrix(y_true, y_pred)
print("Matriz de Confusão:\n", cm)

# Labels das classes
classes = ["Normal", "Ulcerative Colitis", "Polyps", "Esophagitis"]

# Plota matriz de confusão
plt.figure(figsize=(7,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.ylabel('Classe Verdadeira')
plt.xlabel('Classe Predita')
plt.title('Matriz de Confusão - Teste')
plt.show()

# Relatório de classificação completo
print("\nRelatório de Classificação:")
print(classification_report(y_true, y_pred, target_names=classes))
